<a href="https://colab.research.google.com/github/jeffheaton/present/blob/master/WUSTL/CABI-Demand/demand_eda.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Washington University [Olin School of Business](https://olin.wustl.edu/EN-US/Pages/default.aspx)
[Center for Analytics and Business Insights](https://olin.wustl.edu/EN-US/Faculty-Research/research-centers/center-analytics-business-insights/Pages/default.aspx) (CABI)  
[Deep Learning for Demand Forecasting](https://github.com/jeffheaton/present/tree/master/WUSTL/CABI-Demand)  
Copyright 2022 by [Jeff Heaton](https://www.youtube.com/c/HeatonResearch), Released under [CC BY-NC-SA 4.0](https://creativecommons.org/licenses/by-nc-sa/4.0/) 

# Exploratory Data Analysis (EDA)

Connect GDrive, to write out any results.

In [1]:
try:
    from google.colab import drive
    drive.mount('/content/drive', force_remount=True)
    COLAB = True
    print("Note: using Google CoLab")
except:
    print("Note: not using Google CoLab")
    COLAB = False

Mounted at /content/drive
Note: using Google CoLab


Read the provided tabular data CSVs.

In [2]:
import pandas as pd
import os

df_sales = pd.read_csv("https://data.heatonresearch.com/wustl/CABI/demand-forecast/sales_train.csv")
df_items = pd.read_csv("https://data.heatonresearch.com/wustl/CABI/demand-forecast/items.csv")
df_resturant = pd.read_csv("https://data.heatonresearch.com/wustl/CABI/demand-forecast/resturants.csv")

df_sales.date = pd.to_datetime(df_sales.date, errors='coerce') 
df_sales['weekday'] = df_sales.date.dt.day_name()

start_date = df_sales.date.min()
df_sales['week'] = (((df_sales.date - start_date).dt.days)/7).astype(int)

Begin by producing a line graph of all sales over the provided 2-year timespan.

In [3]:
import plotly.express as px

df_plot = df_sales[['date','item_count']].groupby(['date']).sum().reset_index()
fig = px.line(df_plot, x="date", y="item_count", title='Total Sales by Date')
fig.show()

In [4]:
day_order = ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday", "Sunday"]

# Obtain data aggregated by weekday.
df_plot = df_sales[['weekday','item_count']].groupby(['weekday']).sum().reset_index()
# Sort by day of week
df_plot = df_plot.set_index("weekday").loc[day_order].reset_index()

fig = px.bar(df_plot, x='weekday', y='item_count', title='Total Sales by Day of Week')
fig.show()

In [5]:
SINGLE_YEAR = 2020

df_sales_single_year = df_sales[df_sales['date'].dt.year == SINGLE_YEAR]
df_plot = df_sales_single_year[['date','item_count']].groupby(['date']).sum().reset_index() 
fig = px.bar(df_plot, x='date', y='item_count',facet_col_spacing=0, title=f'Total Sales by Day {SINGLE_YEAR}')
fig.update_layout(bargap=0.0,bargroupgap=0.0)
fig.show()

# Most Popular Items

Unique item id's.

In [6]:
df_sales.item_id.unique()

array([  3,   4,  12,  13,  16,  19,  21,  23,  42,  45,  49,  50,  56,
        59,  64,  67,  71,  73,  76,  79,  89,   1,   2,   5,   6,   7,
         8,   9,  10,  11,  14,  15,  17,  18,  20,  22,  24,  25,  26,
        27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,  39,
        40,  41,  43,  44,  46,  47,  48,  51,  52,  53,  54,  55,  57,
        58,  60,  61,  62,  63,  65,  66,  68,  69,  70,  72,  74,  75,
        77,  78,  80,  81,  82,  83,  84,  85,  86,  87,  88,  90,  91,
        92,  93,  94,  95,  96,  97,  98,  99, 100])

Most popular items.

In [7]:
df2 = df_sales[['item_id','item_count']].groupby(['item_id']).sum().reset_index().sort_values('item_count',ascending=False)
df_items2 = df_items[['id','name']]
df_items2.columns = ['item_id','name']
df_popular = df2.merge(df_items2)
print(df_popular.to_string())

    item_id  item_count                                                  name
0        19    162175.0                                    Strawberry Smoothy
1        38     48917.0                                         Mutton Dinner
2         9     41233.0                                          Orange Juice
3        76     38604.0                                    Amazing pork lunch
4        16     31106.0                                  Frozen Milky Smoothy
5        59     28247.0                               Blue Ribbon Beef Entree
6         4     21858.0                       Amazing Steak Dinner with Rolls
7        13     20399.0                               Sweet Frozen Soft Drink
8        56     15172.0                       Sea Bass with Vegetables Dinner
9        67      8092.0                                       Sweet Lamb Cake
10       21      7754.0                     Amazing Fish with Vegetables Meal
11       12      3973.0                                  Fantast

# Individual Item Plots

Plot items per day.

In [8]:
df_plot = df_sales_single_year[['date','item_id','item_count']].groupby(['date','item_id']).sum().reset_index()
df_plot = df_plot.merge(df_items,left_on="item_id",right_on="id")[['date','item_count','name']]
fig = px.bar(df_plot, x='date', y='item_count',color="name", title=f'Item Sales by Date - {SINGLE_YEAR}')
fig.update_layout(bargap=0.0,bargroupgap=0.0)
fig.show()

Plot items per week.

In [9]:
df_plot = df_sales_single_year[['week','item_id','item_count']].groupby(['week','item_id']).sum().reset_index()
df_plot = df_plot.merge(df_items,left_on="item_id",right_on="id")[['week','item_count','name']]
fig = px.bar(df_plot, x='week', y='item_count',color="name", title=f'Total Sales by Week - {SINGLE_YEAR}')
fig.update_layout(bargap=0.0,bargroupgap=0.0)
fig.show()

Plot items per day of week.

In [10]:
day_order = ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday", "Sunday"]

df_plot = df_sales[['weekday','item_id','item_count']].groupby(['weekday','item_id']).sum().reset_index()
df_plot = df_plot.set_index('weekday').loc[day_order].reset_index()
df_plot = df_plot.merge(df_items,left_on="item_id",right_on="id")[['weekday','item_count','name']]
fig = px.bar(df_plot, x='weekday', y='item_count',color="name", title=f'Total Sales by Day of Week - {SINGLE_YEAR}')
fig.update_layout(bargap=0.0,bargroupgap=0.0)
fig.show()